In [1]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

If we want to have **reproducible results**, the environment variable `PYTHONHASHSEED` has to be set to a fixed value, for example to `0`.
Below we check that this environment is set so that results are reproducible.
In order to set this variable we have to use the following sequence of commands in the anaconda shell.  
```
conda activate ai
conda env config vars set PYTHONHASHSEED=0
conda activate ai
```
It is necessary to reactivate the environment `ai` for the setting to take effect.

In [2]:
import os
os.getenv('PYTHONHASHSEED')

'0'

In [3]:
assert hash("test") == 4418353137104490830, "You python hash seed is not correct, results might differ"

In [4]:
import nbimporter

# A Backtracking Solver with Constraint Propagation

## Utility Functions

The module [ast](https://docs.python.org/3.10/library/ast.html) provides the method
`parse` that can be used to create an *abstract syntax tree* for a given expression.

In [5]:
import ast

The function `collect_variables(expr)` takes a string `expr` that can be interpreted as a Python expression as input and collects all variables occurring in `expr`.  It takes care to eliminate the function symbols from the names returned by `extract_variables`.

In [6]:
def collect_variables(expression): 
    tree      = ast.parse(expression)
    Variables = { node.id for node in ast.walk(tree) 
                          if  isinstance(node, ast.Name) 
                          if node.id not in dir(__builtins__)
                }
    return frozenset(Variables)

The function `arb(S)` takes a set `S` as input and returns an arbitrary element from 
this set.

In [7]:
def arb(S):
    for x in S:
        return x

Backtracking is simulated by raising the `Backtrack` exception.  We define this new class of exceptions so that we can distinguish `Backtrack` exceptions from ordinary exceptions.  This is done by creating a new, empty class that is derived from the class `Exception`.  

In [8]:
class Backtrack(Exception):
    pass

Given a list of sets `L`, the function `union(L)` returns the set of all elements occurring in some set $S$ that is itself a member of the list `L`, i.e. we have
$$ \texttt{union}(L) = \{ x \mid \exists S \in L : x \in L \}. $$ 

In [9]:
def union(L):
    return { x for S in L
               for x in S
           }

In [10]:
union([ {1, 2}, {'a', 'b'}, {1, 'a'} ])

{1, 2, 'a', 'b'}

## The Constraint Propagation Solver

The procedure `solve(P, lcv=False)` takes two arguments:
* `P` is a *constraint satisfaction problem*, i.e. `P` is a triple of the form 
   $$ \mathtt{P} = \langle \mathtt{Variables}, \mathtt{Values}, \mathtt{Constraints} \rangle $$
   where 
   - `Variables` is a set of strings which serve as *variables*,
   - `Values` is a set of *values* that can be assigned to the variables in the set `Variables`.
   - `Constraints` is a set of *formulas* from first order logic.  
     Each of these formulas is  called a *constraint* of `P`.
     The formulas are represented as strings.
* `lcv` is a Boolean flag.  If this flag is set to `True`, the *least constraining value* heuristic is used when choosing values. 
  Otherwise, the values are chosen arbitrarily.   

Initially, the function `solve` checks that the set `Constraints` does not contain any variables that are not 
elements of the set `Variables`.  Furthermore, it checks that all variables in `Variables` do indeed occur 
in one of the constraints.  These two checks are useful to capture spelling mistakes.

Then, the function `solve` converts the CSP `P` into an *augmented CSP* where every constraint $f$ is annotated with the variables occurring in $f$.  

The most important data structure maintained by `solve` is the dictionary `ValuesPerVar`.  For every variable $x$ occurring in a constraint of `P`, the expression $\texttt{ValuesPerVar}[x]$ is the set of values that can be used to instantiate the variable $x$.  Initially, 
$\texttt{ValuesPerVar}[x]$ is set to `Values`, but as the search for a solution proceeds, the sets $\texttt{ValuesPerVar}[x]$ are reduced by removing any values that cannot be part of a solution.
This way, the consequences of binding one variable to a value are *propagated* to the other variables.

Next, the function `solve` divides the constraints into two groups:
- The *unary* constraints are those that only contain a single variable.

  The unary constraints can be solved immediately: 
  If $f$ is a unary constraint containing only the variable $x$, the set $\texttt{ValuesPerVar}(x)$ 
  is reduced to the set of those values $v$ such that $\texttt{eval}(f, \{x\mapsto v\})$ is true.
- The remaining constraints contain at least two different variables.

After the unary constraints have been taken care of, `backtrack_search` is called to solve the remaining constraint satisfaction problem.  The function `backtrack_search` uses both *backtracking* and *constraint propagation* to solve the remaining constraints.
Furthermore, the *most constrained variable* heuristic and, if `lvc` is set to `True`, the *least constraining value* heuristic are used.

In [11]:
def solve(P, lcv=False):
    Variables, Values, Constraints = P 
    VarsInConstrs  = union([ collect_variables(f) for f in Constraints ])
    MisspelledVars = (VarsInConstrs - Variables) | (Variables - VarsInConstrs)
    if len(MisspelledVars) > 0:
        print('Did you misspell any of the following Variables?')
        for v in MisspelledVars:
            print(v)
    Annotated    = { (f, collect_variables(f)) for f in Constraints }
    ValuesPerVar = { v: Values for v in Variables }
    UnaryConstrs = { (f, V) for f, V in Annotated if  len(V) == 1 }
    OtherConstrs = { (f, V) for f, V in Annotated if  len(V) >= 2 }
    try:
        for f, V in UnaryConstrs:
            var = arb(V)
            ValuesPerVar[var] = solve_unary(f, var, ValuesPerVar[var])
        return backtrack_search({}, ValuesPerVar, OtherConstrs, lcv)
    except Backtrack:
        return None

The function `solve_unary` takes three arguments:
* `f` is a unary constraint, i.e. a constraint that contains only one variable,
* `x` is the variable occurring in `f`, and 
* `Values` is the set of values that can be assigned to the variable `x`.  

The function returns the subset of those values `v` from the set `Values` that can be substituted for `x` such that $\texttt{eval}(f, \{ x \mapsto v \})$ evaluates as `True`.  If the unary constraint `f` is unsolvable, then the given CSP is unsolvable and an exception is raised. 

In [12]:
def solve_unary(f, x, Values):
    Legal = { value for value in Values if eval(f, { x: value }) }
    if len(Legal) == 0:
        raise Backtrack()
    return Legal

The function `backtrack_search` takes four arguments:
- `Assignment` is a partial variable assignment that is represented as a
   dictionary.  Initially, this assignment will be the  empty dictionary.     
   Every recursive call of `backtrack_search` adds the assignment of one 
   variable to  the given assignment. 
   
   `Assignment` is *partially consistent*: If a constraint `F` can be evaluated with `Assignment`, then `eval(F, Assignment)`must yield true. 
- `ValuesPerVar` is a dictionary.  For every variable `x`, `ValuesPerVar[x]` is the set of values 
   that still might be assigned to `x`.
- `Constraints` is a set of pairs of the form `(F, V)` where `F` is a constraint and `V` is the 
   set of variables occurring in `V`.
- `lcv` is a Boolean flag.  If this flag is set to true, the *least constraining value* heuristic is used.

The function tries to solve the given CSP via backtracking.  Instead of picking the variables arbitrarily, it uses 
the *most constraint variable* heuristic and therefore instantiates those variables first, that have the least
remaining values.  This way, a dead end in the search is discovered sooner.

In [13]:
def backtrack_search(Assignment, ValuesPerVar, Constraints, lcv):
    if len(Assignment) == len(ValuesPerVar):
        return Assignment
    x = most_constrained_variable(Assignment, ValuesPerVar)
    if lcv and len(ValuesPerVar[x]) > 1:
        ValueList = least_constraining(x, ValuesPerVar, Assignment, Constraints)
    else:
        ValueList = ValuesPerVar[x]
    for v in ValueList: 
        try:
            NewValues    = propagate(x, v, Assignment, Constraints, ValuesPerVar)
            NewAssign    = Assignment.copy()
            NewAssign[x] = v
            return backtrack_search(NewAssign, NewValues, Constraints, lcv)
        except Backtrack:
            continue
    raise Backtrack()

The function `most_constrained_variable` takes two parameters:
- `Assignment` is a *partial variable assignment* that assigns values to variables.  It is represented as a dictionary.
- `ValuesPerVar` is a dictionary that maps variables to the set of values that may be assigned to these variables,
  i.e. for every variable `x`, `ValuesPerVar[x]` is the set of values that can be assigned to the variable `x`
  without violating a constraint.
  
The function returns an unassigned variable `x` such that the number of values in `ValuesPerVar[x]` is minimal among all other unassigned variables.

In [14]:
def most_constrained_variable(Assignment, ValuesPerVar):
    Unassigned = { (x, len(U)) for x, U in ValuesPerVar.items()
                               if  x not in Assignment
                 }
    minSize = min(lenU for _, lenU in Unassigned)
    return arb({ x for x, lenU in Unassigned if lenU == minSize })

We import `math` because this gives us access to the infinite value $\infty$, which is available as `math.inf`. 

In [15]:
import math

The function `least_constraining` takes four arguments:
* `x` is a variable. 
* `ValuesPerVar` is a dictionary.  For every variable `var`, `ValuesPerVar[var]` is the set of values that can be assigned to `var`.
* `Assignment` is a partial variable assignment.
* `Constraints` is a set of annotated constraints.

This function returns a list of values that can be substituted for the variable `x`.  
This list is sorted so that the *least constraining* values are at the beginning of this list. 

In [16]:
def least_constraining(x, ValuesPerVar, Assignment, Constraints):
    NumbersValues = []
    for value in ValuesPerVar[x]:
        ReducedValues = ValuesPerVar.copy()
        num_removed = shrinkage(x, value, Assignment, ReducedValues, Constraints)
        if num_removed != math.inf:
            NumbersValues.append( (num_removed, value) )
    NumbersValues.sort(key=lambda p: p[0])
    return [val for _, val in NumbersValues]

The function `shrinkage` takes 5 arguments:
- `x` is a variable that has not yet been assigned a value.
- `value` is a value that is to be assigned to the variable `x`.
- `Assignment` is a partial variable assignment that does not assign a value to `x`.
- `ValuesPerVar` is a dictionary that has variables as keys.  For every variable `z`, `ValuesPerVar[z]` is the set of values that 
  can still be assigned to the variable `z`.
- `Constraints` is a set of pairs of the form `(f, V)` where `f` is a constraint and `V` is the set of variables occurring in `f`.

This function returns the *shrinkage number*, which is the number of values that need to be removed from the set 
`ValuesPerVar[y]` for those variables `y` that are different from `x` if we assign `value` to the variable `x`. 
If the assignment `{ x: value }` results in any of the sets `ValuesPerVar[y]`
becoming empty, then the function returns `math.inf` in order to signal that the assignment `{ x: value }` leads to an unsolvable problem.

In [17]:
def shrinkage(x, value, Assignment, ValuesPerVar, Constraints):
    count     = 0   # number of values removed from ValuesPerVar
    BoundVars = set(Assignment.keys())
    for f, Vars in Constraints:
        if x in Vars:
            UnboundVars = Vars - BoundVars - { x }
            if len(UnboundVars) == 1:
                y = arb(UnboundVars)
                Legal = set()
                for w in ValuesPerVar[y]:
                    NewAssign    = Assignment.copy()
                    NewAssign[x] = value
                    NewAssign[y] = w
                    if eval(f, NewAssign):
                        Legal.add(w)
                    else:
                        count += 1
                if len(Legal) == 0:
                    return math.inf
                ValuesPerVar[x] = Legal
    return count           

The function `propagate` takes five arguments:
- `x` is a variable,
- `v` is a value that is supposed to be assigned to `x`.
- `Assignment` is a partial assignment that contains assignments for variables that are different from `x`.
- `Constraints` is a set of annotated constraints.
- `ValuesPerVar` is a dictionary assigning sets of values to all variables.  For every unassigned variable `z`,  `ValuesPerVar[z]` is the set of values that still might be assigned to `z`.

The purpose of the function  `propagate` is to compute how the sets `ValuesPerVar[z]` can be shrunk when the value `v` is assigned to the variable `x`.  The dictionary `ValuesPerVar` with appropriately reduced sets `ValuesPerVar[z]` is returned.  In particular, the consequences of assigning the value `v` to the variable `x` are *propagated*:
If there is a constraint `f` such that `x` occurs in `f` and there is just one variable `y` left that occurs in 
`f` and that is not yet bound in `Assignment`, then the values that can still be assigned to `y` are computed
and the dictionary `ValuesDict` is updated accordingly.  If there are no values left that can be assigned to 
`y` without violating the constraint `f`, the function backtracks.

In [18]:
def propagate(x, v, Assignment, Constraints, ValuesPerVar):
    ValuesDict    = ValuesPerVar.copy()
    ValuesDict[x] = { v }
    BoundVars     = set(Assignment.keys())
    for f, Vars in Constraints:
        if x in Vars:
            UnboundVars = Vars - BoundVars - { x }
            if len(UnboundVars) == 1:
                y = arb(UnboundVars)
                Legal = set()
                for w in ValuesDict[y]:
                    NewAssign = Assignment.copy()
                    NewAssign[x] = v
                    NewAssign[y] = w
                    if eval(f, NewAssign):
                        Legal.add(w)
                if not Legal:
                    raise Backtrack()
                ValuesDict[y] = Legal
    return ValuesDict

## Solving the *Eight-Queens-Puzzle*

In [19]:
from NQueensProblemCSP import create_csp, show_solution

In [20]:
P = create_csp(8)

Constraint Propagation with the *least constraining value heuristic* takes about 27 milliseconds on my Windows desktop to solve the eight queens puzzle.

In [21]:
%%time
Solution = solve(P, lcv=True)
print(f'Solution = {Solution}')

Solution = {'V1': 1, 'V6': 2, 'V3': 4, 'V7': 5, 'V5': 8, 'V4': 6, 'V2': 7, 'V8': 3}
CPU times: user 35.9 ms, sys: 2.66 ms, total: 38.5 ms
Wall time: 37 ms


In [22]:
show_solution(Solution)

Constraint Propagation without the *least constraining value heuristic* takes only 13 milliseconds on my desktop to solve the eight queens puzzle.

In [23]:
%%time
Solution = solve(P)
print(f'Solution = {Solution}')

Solution = {'V1': 1, 'V6': 2, 'V3': 4, 'V7': 5, 'V5': 8, 'V4': 6, 'V2': 7, 'V8': 3}
CPU times: user 15.6 ms, sys: 477 µs, total: 16.1 ms
Wall time: 16 ms


In [24]:
P = create_csp(32)

Constraint propagation can solve the 32 queens problem in less than $2.2$ seconds, if the *least constraining value heuristic* is used.

In [25]:
%%time
Solution = solve(P, True)
print(f'Solution = {Solution}')

Solution = {'V14': 1, 'V11': 2, 'V16': 5, 'V26': 3, 'V17': 32, 'V25': 6, 'V1': 8, 'V2': 25, 'V5': 18, 'V6': 11, 'V7': 4, 'V8': 17, 'V15': 26, 'V13': 7, 'V4': 29, 'V19': 9, 'V3': 13, 'V10': 24, 'V18': 21, 'V9': 10, 'V21': 16, 'V20': 28, 'V22': 20, 'V12': 14, 'V29': 22, 'V30': 15, 'V23': 23, 'V27': 31, 'V32': 12, 'V24': 30, 'V31': 19, 'V28': 27}
CPU times: user 2.3 s, sys: 5.47 ms, total: 2.31 s
Wall time: 2.31 s


In [26]:
show_solution(Solution, "60%")

Constraint propagation can solve the 32 queens problem in less than $264$ milliseconds, if the *least constraining value heuristic* is not used.
The $n$-queens problem is a relatively easy CSP and hence the *least constraining value* is not useful.

In [27]:
%%time
Solution = solve(P)
print(f'Solution = {Solution}')

Solution = {'V14': 1, 'V11': 2, 'V16': 4, 'V26': 3, 'V28': 6, 'V4': 5, 'V30': 9, 'V29': 32, 'V22': 8, 'V27': 10, 'V32': 12, 'V31': 7, 'V15': 11, 'V12': 16, 'V13': 18, 'V9': 15, 'V7': 14, 'V18': 17, 'V8': 24, 'V10': 30, 'V21': 21, 'V6': 19, 'V24': 26, 'V1': 22, 'V3': 13, 'V23': 20, 'V25': 23, 'V5': 31, 'V20': 29, 'V17': 28, 'V19': 27, 'V2': 25}
CPU times: user 234 ms, sys: 4.15 ms, total: 238 ms
Wall time: 238 ms


In [29]:
show_solution(Solution, "60%")

## Solving the *Zebra Puzzle*

In [30]:
from Zebra import zebra_csp, show_solution

In [31]:
zebra = zebra_csp()

Constraint propagation with the *least constraining value* heuristic takes about 12 milliseconds to solve the *Zebra Puzzle*.

In [32]:
%%time
Solution = solve(zebra, True)

CPU times: user 19.3 ms, sys: 1.61 ms, total: 20.9 ms
Wall time: 20.3 ms


In [33]:
show_solution(Solution)

House,Nationality,Drink,Animal,Brand,Colour
1,Norwegian,Water,Fox,Kools,Yellow
2,Ukrainian,Tea,Horse,Chesterfields,Blue
3,English,Milk,Snails,OldGold,Red
4,Spanish,OrangeJuice,Dog,LuckyStrike,Ivory
5,Japanese,Coffee,Zebra,Parliaments,Green


If the *least constraining value* heuristic is not used, it takes about 12 milliseconds to solve the *Zebra Puzzle*.

In [34]:
%%time
Solution = solve(zebra)

CPU times: user 17.4 ms, sys: 817 µs, total: 18.2 ms
Wall time: 17.9 ms


## Solving a Sudoku Puzzle

In [35]:
from Sudoku import sudoku_csp, show_sudoku, find_alternative

In [36]:
Sudoku = [ ["*",  3 ,  9 , "*", "*", "*", "*", "*",  7 ], 
           ["*", "*", "*",  7 , "*", "*",  4 ,  9 ,  2 ],
           ["*", "*", "*", "*",  6 ,  5 , "*",  8 ,  3 ],
           ["*", "*", "*",  6 , "*",  3 ,  2 ,  7 , "*"],
           ["*", "*", "*", "*",  4 , "*",  8 , "*", "*"],
           [ 5 ,  6 , "*", "*", "*", "*", "*", "*", "*"],
           ["*", "*",  5 ,  2 , "*",  9 , "*", "*",  1 ],
           ["*",  2 ,  1 , "*", "*", "*", "*",  4 , "*"],
           [ 7 , "*", "*", "*", "*", "*",  5 , "*", "*"]
         ]

In [37]:
csp = sudoku_csp(Sudoku)
csp

({'V11',
  'V12',
  'V13',
  'V14',
  'V15',
  'V16',
  'V17',
  'V18',
  'V19',
  'V21',
  'V22',
  'V23',
  'V24',
  'V25',
  'V26',
  'V27',
  'V28',
  'V29',
  'V31',
  'V32',
  'V33',
  'V34',
  'V35',
  'V36',
  'V37',
  'V38',
  'V39',
  'V41',
  'V42',
  'V43',
  'V44',
  'V45',
  'V46',
  'V47',
  'V48',
  'V49',
  'V51',
  'V52',
  'V53',
  'V54',
  'V55',
  'V56',
  'V57',
  'V58',
  'V59',
  'V61',
  'V62',
  'V63',
  'V64',
  'V65',
  'V66',
  'V67',
  'V68',
  'V69',
  'V71',
  'V72',
  'V73',
  'V74',
  'V75',
  'V76',
  'V77',
  'V78',
  'V79',
  'V81',
  'V82',
  'V83',
  'V84',
  'V85',
  'V86',
  'V87',
  'V88',
  'V89',
  'V91',
  'V92',
  'V93',
  'V94',
  'V95',
  'V96',
  'V97',
  'V98',
  'V99'},
 {1, 2, 3, 4, 5, 6, 7, 8, 9},
 {'V11 != V12',
  'V11 != V13',
  'V11 != V14',
  'V11 != V15',
  'V11 != V16',
  'V11 != V17',
  'V11 != V18',
  'V11 != V19',
  'V11 != V21',
  'V11 != V22',
  'V11 != V23',
  'V11 != V31',
  'V11 != V32',
  'V11 != V33',
  'V11 != V41',


Constraint propagation with the *least constraining value* heuristic takes about 112 milliseconds to solve 
the given sudoku.

In [38]:
%%time
Solution = solve(csp, True)

CPU times: user 109 ms, sys: 3.11 ms, total: 112 ms
Wall time: 110 ms


In [39]:
show_sudoku(Solution, Sudoku)

|| 1 | 3 | 9 || 4 | 2 | 8 || 6 | 5 | 7 || 
-----------------------------------------
|| 6 | 5 | 8 || 7 | 3 | 1 || 4 | 9 | 2 || 
-----------------------------------------
|| 2 | 4 | 7 || 9 | 6 | 5 || 1 | 8 | 3 || 
|| 8 | 1 | 4 || 6 | 9 | 3 || 2 | 7 | 5 || 
-----------------------------------------
|| 9 | 7 | 3 || 5 | 4 | 2 || 8 | 1 | 6 || 
-----------------------------------------
|| 5 | 6 | 2 || 1 | 8 | 7 || 9 | 3 | 4 || 
|| 4 | 8 | 5 || 2 | 7 | 9 || 3 | 6 | 1 || 
-----------------------------------------
|| 3 | 2 | 1 || 8 | 5 | 6 || 7 | 4 | 9 || 
-----------------------------------------
|| 7 | 9 | 6 || 3 | 1 | 4 || 5 | 2 | 8 || 


Constraint propagation without the *least constraining value* heuristic takes 144 milliseconds to solve 
the given sudoku. Hence, in this case the *least constraining value* heuristic is useful. 

In [40]:
%%time
Solution = solve(csp, False)

CPU times: user 133 ms, sys: 2.05 ms, total: 135 ms
Wall time: 134 ms


Let's check whether the solution is unique.

In [41]:
csp = find_alternative(csp, Solution)
csp

({'V11',
  'V12',
  'V13',
  'V14',
  'V15',
  'V16',
  'V17',
  'V18',
  'V19',
  'V21',
  'V22',
  'V23',
  'V24',
  'V25',
  'V26',
  'V27',
  'V28',
  'V29',
  'V31',
  'V32',
  'V33',
  'V34',
  'V35',
  'V36',
  'V37',
  'V38',
  'V39',
  'V41',
  'V42',
  'V43',
  'V44',
  'V45',
  'V46',
  'V47',
  'V48',
  'V49',
  'V51',
  'V52',
  'V53',
  'V54',
  'V55',
  'V56',
  'V57',
  'V58',
  'V59',
  'V61',
  'V62',
  'V63',
  'V64',
  'V65',
  'V66',
  'V67',
  'V68',
  'V69',
  'V71',
  'V72',
  'V73',
  'V74',
  'V75',
  'V76',
  'V77',
  'V78',
  'V79',
  'V81',
  'V82',
  'V83',
  'V84',
  'V85',
  'V86',
  'V87',
  'V88',
  'V89',
  'V91',
  'V92',
  'V93',
  'V94',
  'V95',
  'V96',
  'V97',
  'V98',
  'V99'},
 {1, 2, 3, 4, 5, 6, 7, 8, 9},
 {'V11 != V12',
  'V11 != V13',
  'V11 != V14',
  'V11 != V15',
  'V11 != V16',
  'V11 != V17',
  'V11 != V18',
  'V11 != V19',
  'V11 != V21',
  'V11 != V22',
  'V11 != V23',
  'V11 != V31',
  'V11 != V32',
  'V11 != V33',
  'V11 != V41',


In [42]:
%%time
Solution = solve(csp)
if Solution:
    print('There is another solution.')
else:
    print('The solution is unique!')

The solution is unique!
CPU times: user 138 ms, sys: 2.17 ms, total: 140 ms
Wall time: 139 ms


## Solving the Crypto-Arithmetic Puzzle

In [43]:
from CryptoArithmetic import crypto_csp, crypto_csp_hard, show_solution

In [44]:
csp = crypto_csp()
csp

({'C1', 'C2', 'C3', 'D', 'E', 'M', 'N', 'O', 'R', 'S', 'Y'},
 {0, 1, 2, 3, 4, 5, 6, 7, 8, 9},
 {'(D + E)      % 10 == Y',
  '(D + E)      // 10 == C1',
  '(E + O + C2) % 10 == N',
  '(E + O + C2) // 10 == C3',
  '(N + R + C1) % 10 == E',
  '(N + R + C1) // 10 == C2',
  '(S + M + C3) % 10 == O',
  '(S + M + C3) // 10 == M',
  'D != E',
  'D != M',
  'D != N',
  'D != O',
  'D != R',
  'D != S',
  'D != Y',
  'E != M',
  'E != N',
  'E != O',
  'E != R',
  'E != S',
  'E != Y',
  'M != 0',
  'M != N',
  'M != O',
  'M != R',
  'M != S',
  'M != Y',
  'N != O',
  'N != R',
  'N != S',
  'N != Y',
  'O != R',
  'O != S',
  'O != Y',
  'R != S',
  'R != Y',
  'S != 0',
  'S != Y'})

Constraint propagation takes about 118 milliseconds to solve the crypto-arithmetic puzzle if the 
*least constraining value* heuristic is used.

In [45]:
%%time
Solution = solve(csp, True)

CPU times: user 154 ms, sys: 1.84 ms, total: 156 ms
Wall time: 161 ms


In [46]:
show_solution(Solution)

O = 0
R = 8
M = 1
Y = 2
N = 6
D = 7
E = 5
S = 9

The solution of

    S E N D
  + M O R E
  ---------
  M O N E Y

is as follows

    9 5 6 7
  + 1 0 8 5
  1 0 6 5 2


Constraint propagation takes about 1.1 seconds if the *least constraining value* heuristic is not used.

In [47]:
%%time
Solution = solve(csp)

CPU times: user 1.25 s, sys: 4.26 ms, total: 1.25 s
Wall time: 1.25 s


Let us try the hard version of the puzzle.

In [48]:
csp = crypto_csp_hard()

In [49]:
%%time
Solution = solve(csp, True)

CPU times: user 1.06 s, sys: 3.84 ms, total: 1.07 s
Wall time: 1.07 s


In [50]:
%%time
Solution = solve(csp)

CPU times: user 903 ms, sys: 3.47 ms, total: 906 ms
Wall time: 906 ms
